# This notebook is heavily inspired by: *https://www.kaggle.com/jhoward/improved-lstm-baseline-glove-dropout-lb-0-048*

In [43]:
import sys 
import os
import re
import codecs

import numpy as np
import pandas as pd

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D, CuDNNGRU
from keras.layers import Bidirectional, GlobalMaxPool1D, MaxPooling1D
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras import initializers, regularizers, constraints, optimizers, layers

In [3]:
#basic config parameters
embed_size = 50
max_features = 20000
maxlen = 100

# Preprocessing _(at least atm) is leading to worse results...

In [5]:
def text_preprocessing(text):
    from nltk.corpus import stopwords
    from nltk.tokenize import RegexpTokenizer
    from nltk.stem import WordNetLemmatizer
    
    tokenizer = RegexpTokenizer(r'\w+')
    stop = set(stopwords.words('english')) 
    lemmatizer = WordNetLemmatizer()
     
    processed = []   
    print("Start preprocessing: ")
    for i in range(len(text)):
        if i%10000 == 0:
            print("{} of {}".format(i, len(text)))
        temp = tokenizer.tokenize(text.iloc[i])
        temp = [lemmatizer.lemmatize(i) for i in temp if i not in stop]
        temp = (" ").join(temp)
        processed.append(temp)
        
    return processed

In [6]:
#Read data and replace missing values

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

train = train.fillna("_na_")
test = test.fillna("_na_")
train["processed"] = text_preprocessing(train["comment_text"])
test["processed"] = text_preprocessing(test["comment_text"])

list_sentences_train = train["processed"].values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes]
list_sentences_test = test["proceessed"].values

Start preprocessing: 
0 of 159571
10000 of 159571
20000 of 159571
30000 of 159571
40000 of 159571
50000 of 159571
60000 of 159571
70000 of 159571
80000 of 159571
90000 of 159571
100000 of 159571
110000 of 159571
120000 of 159571
130000 of 159571
140000 of 159571
150000 of 159571
Start preprocessing: 
0 of 153164
10000 of 153164
20000 of 153164
30000 of 153164
40000 of 153164
50000 of 153164
60000 of 153164
70000 of 153164
80000 of 153164
90000 of 153164
100000 of 153164
110000 of 153164
120000 of 153164
130000 of 153164
140000 of 153164
150000 of 153164


KeyError: 'proceessec'

In [8]:
list_sentences_train2 = train["comment_text"].values
list_sentences_test2 = test["comment_text"].values

<p>Standard Keras preprocessing, each comment is turned into a list of words indexes of equal length (including truncation/padding)</p>

In [37]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer

TfidfVec = TfidfVectorizer(ngram_range=(1,4))

TfidfVec.fit(pd.concat([train["comment_text"], test["comment_text"]], axis=0))
tfidf_train = TfidfVec.transform(train["comment_text"])
tfidf_test = TfidfVec.transform(train["comment_text"])

In [31]:
tfidf_train[0].shape

(1, 28136869)

In [9]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train2))
list_tokenized_train2 = tokenizer.texts_to_sequences(list_sentences_train2)
list_tokenized_test2 = tokenizer.texts_to_sequences(list_sentences_test2)
X_t2 = pad_sequences(list_tokenized_train2, maxlen=maxlen)
X_te2 = pad_sequences(list_tokenized_test2, maxlen=maxlen)

In [10]:
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(0,1,(nb_words, embed_size))

<p>Simple bidirectional LSTM with two fully connected layers</p>

In [36]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(GRU(100, return_sequences=True, dropout=0.15, recurrent_dropout=0.15))(x)
x = Bidirectional(GRU(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dense(50, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(6, activation='sigmoid')(x)
model2 = Model(inputs=inp, outputs=x)
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [39]:
def get_model(embedding_matrix, sequence_length, dropout_rate, recurrent_units, dense_size):
    input_layer = Input(shape=(sequence_length,))
    embedding_layer = Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1],
                                weights=[embedding_matrix], trainable=False)(input_layer)
    x = Bidirectional(CuDNNGRU(recurrent_units, return_sequences=True))(embedding_layer)
    x = Dropout(dropout_rate)(x)
    x = Bidirectional(CuDNNGRU(recurrent_units, return_sequences=False))(x)
    x = Dense(dense_size, activation="relu")(x)
    output_layer = Dense(6, activation="sigmoid")(x)

    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(loss='binary_crossentropy',
                  optimizer=RMSprop(clipvalue=1, clipnorm=1),
metrics=['accuracy'])

In [44]:
get_model(embedding_matrix=embedding_matrix, sequence_length=maxlen, dropout_rate=0.3, recurrent_units=64, dense_size=32)

NotFoundError: Op type not registered 'CudnnRNN'

In [33]:
filepath = r"C:\WM\ToxicComments"

callbacks = [
    EarlyStopping(monitor = 'loss', min_delta=0, patience=1, verbose=10, mode='auto'),
    ModelCheckpoint(filepath=filepath, monitor = 'loss', verbose=1, save_best_only=True),
    TensorBoard(log_dir=filepath)
]

model2.fit(X_t, y, batch_size=32, epochs=10, callbacks=callbacks, validation_split=0.1)

ValueError: Error when checking input: expected input_5 to have shape (100,) but got array with shape (28136869,)

In [37]:
filepath = r"C:\WM\ToxicComments"
save = r"C:\WM\ToxicComments\save.hd5f"

callbacks = [
    EarlyStopping(monitor = 'loss', min_delta=0, patience=1, verbose=10, mode='min'),
    ModelCheckpoint(filepath=save, monitor = 'loss', verbose=1, save_best_only=True),
    #TensorBoard(log_dir=filepath)
]

model2.fit(X_t2, y, batch_size=128, epochs=10, callbacks=callbacks, validation_split=0.1)

Train on 143613 samples, validate on 15958 samples
Epoch 1/10
143488/143613 [============================>.] - ETA: 1s - loss: 0.0866 - acc: 0.9737
Epoch 00001: loss improved from inf to 0.08658, saving model to C:\WM\ToxicComments\save.hd5f
143613/143613 [==============================] - 1191s 8ms/step - loss: 0.0866 - acc: 0.9737 - val_loss: 0.0611 - val_acc: 0.9794
Epoch 2/10
143488/143613 [============================>.] - ETA: 1s - loss: 0.0574 - acc: 0.9804
Epoch 00002: loss improved from 0.08658 to 0.05739, saving model to C:\WM\ToxicComments\save.hd5f
143613/143613 [==============================] - 1283s 9ms/step - loss: 0.0574 - acc: 0.9805 - val_loss: 0.0546 - val_acc: 0.9810
Epoch 3/10
143488/143613 [============================>.] - ETA: 1s - loss: 0.0509 - acc: 0.9819
Epoch 00003: loss improved from 0.05739 to 0.05090, saving model to C:\WM\ToxicComments\save.hd5f
143613/143613 [==============================] - 1302s 9ms/step - loss: 0.0509 - acc: 0.9819 - val_loss: 0.0

In [38]:
#y_test = model.predict([X_te], batch_size=1024, verbose=1)
y_test2 = model2.predict([X_te2], batch_size=1024, verbose=1)
#sample_submission = pd.read_csv("sample_submission.csv")
#sample_submission[list_classes] = y_test
#sample_submission.to_csv("submission_LSTM3.csv", index=False)
sample_submission2 = pd.read_csv("sample_submission.csv")
sample_submission2[list_classes] = y_test2
sample_submission2.to_csv("submission_LSTM4.csv", index=False)

153164/153164 [==============================] - 203s 1ms/step
